In [63]:
import pandas as pd
import numpy as np
import scipy.stats as stats

## Data Preparation

In [2]:
outpatient = pd.read_csv('./VTOUTP16.TXT')

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (2,5,54,55,56,57) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
ra = pd.read_csv('RA.csv')
other_ra = pd.read_csv('Other_RA.csv')

## Step 1: Identify the RA cohort using the outpatient file. 

In [20]:
ra_outpatient = pd.DataFrame()
for i in range(20):
    temp = outpatient.merge(ra['ICD-10 Codes'], left_on= f'DX{i+1}', right_on='ICD-10 Codes')
    ra_outpatient = pd.concat([ra_outpatient, temp])

In [21]:
other_ra_outpatient = pd.DataFrame()
for i in range(20):
    temp = outpatient.merge(other_ra['ICD-10 Codes'], left_on= f'DX{i+1}', right_on='ICD-10 Codes')
    other_ra_outpatient = pd.concat([other_ra_outpatient, temp])

In [28]:
print(ra_outpatient.shape)
ra_outpatient.drop_duplicates(subset='Uniq').shape
# 981 Diagnoses  976 patients

(981, 71)


(976, 71)

In [29]:
print(other_ra_outpatient.shape)
other_ra_outpatient.drop_duplicates(subset='Uniq').shape
# 31 diagnoses 30 patients

(31, 71)


(30, 71)

## Step 2: Identify the most common types of the RA

In [39]:
ra_outpatient_freq = ra_outpatient.groupby('ICD-10 Codes')['Uniq'].count()
ra_outpatient_freq.sort_values(ascending=False)
# Top 3 M069. M0579, M059

ICD-10 Codes
M069      909
M0579      17
M059        8
M0600       6
M0609       4
M06871      4
M06072      3
M0570       2
M06342      2
M06341      2
M06322      2
M0689       2
M06071      2
M0680       2
M06851      2
M06861      2
M06872      1
M05741      1
M05742      1
M05761      1
M05762      1
M06832      1
M05812      1
M06031      1
M06051      1
M06842      1
M06331      1
M06041      1
Name: Uniq, dtype: int64

In [38]:
other_ra_outpatient_freq = other_ra_outpatient.groupby('ICD-10 Codes')['Uniq'].count()

other_ra_outpatient_freq.sort_values(ascending=False)

# Top 3 M510, M05671, M0519

ICD-10 Codes
M0510     21
M05671     2
M0519      2
M061       1
M05672     1
M0560      1
M05142     1
M05141     1
M0500      1
Name: Uniq, dtype: int64

## Step 3: Gender differences in RA

In [44]:
ra_outpatient['is_RA'] = 1
outpatient_new = outpatient.merge(ra_outpatient[['Uniq', 'is_RA']].drop_duplicates(subset='Uniq'), how='left').fillna(0)

In [45]:
other_ra_outpatient['is_other_RA'] = 1
outpatient_new = outpatient_new.merge(other_ra_outpatient[['Uniq', 'is_other_RA']].drop_duplicates(subset='Uniq'), how='left').fillna(0)

In [53]:
outpatient_new.sex.replace(' ', np.nan, inplace=True)

In [56]:
outpatient_new.dropna(subset=['sex', 'is_RA', 'is_other_RA'], inplace=True)

In [61]:
outpatient_new.sex = outpatient_new.sex.astype(int)
outpatient_new.is_RA = outpatient_new.is_RA.astype(int)
outpatient_new.is_other_RA = outpatient_new.is_other_RA.astype(int)

### Gender bias in RA

In [64]:
ra_xtab = outpatient_new.groupby(['sex', 'is_RA'])['Uniq'].count().unstack()
oddsratio, pvalue = stats.fisher_exact(ra_xtab)
print('Odds ratio:', oddsratio)
print('p-value: ', pvalue)

Odds ratio: 2.3975543884127677
p-value:  1.4457910557520762e-36


In [65]:
ra_xtab

is_RA,0,1
sex,,
1,168150,252
2,201496,724


### Gender bias in other_RA

In [67]:
other_ra_xtab = outpatient_new.groupby(['sex', 'is_other_RA'])['Uniq'].count().unstack()
oddsratio, pvalue = stats.fisher_exact(other_ra_xtab)
print('Odds ratio:', oddsratio)
print('p-value: ', pvalue)

Odds ratio: 0.9517290883040325
p-value:  1.0


In [68]:
other_ra_xtab

is_other_RA,0,1
sex,,
1,168388,14
2,202204,16


## Step 4: Calculate the inter-quartile range of the costs

In [71]:
outpatient.CHRGS.describe()

count    370633.000000
mean       3087.405855
std        4915.324061
min           0.000000
25%         682.480000
50%        1521.620000
75%        3440.180000
max      227311.780000
Name: CHRGS, dtype: float64

In [72]:
stats.iqr(outpatient.CHRGS)

2757.7

## Step 5: Study of service utilization

In [73]:
revcode = pd.read_csv('VTREVCODE16.TXT')

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (6,7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [76]:
revcode_ra = revcode.merge(ra_outpatient.drop_duplicates(subset='Uniq')['Uniq'])

In [77]:
revcode_other_ra = revcode.merge(other_ra_outpatient.drop_duplicates(subset='Uniq')['Uniq'])

In [80]:
revcode_ra.groupby('REVCODE')['Uniq'].count().sort_values(ascending=False)
# TOP 5 Services 300, 636, 250, 450, 320

REVCODE
300    3219
636    1218
250    1161
450    1055
320     373
259     286
730     280
270     234
360     229
370     219
710     215
260     199
272     194
762     163
301     155
420     153
258     153
310     139
361     111
637     105
510      82
351      78
460      75
761      74
278      73
352      70
305      69
350      62
324      48
490      41
       ... 
307      10
434       9
610       8
920       7
255       6
480       6
612       6
341       5
390       5
335       4
311       4
309       4
481       4
731       4
444       3
401       2
292       2
312       2
391       2
615       2
623       2
280       1
279       1
275       1
359       1
619       1
681       1
740       1
790       1
942       1
Name: Uniq, Length: 81, dtype: int64

In [81]:
revcode_other_ra.groupby('REVCODE')['Uniq'].count().sort_values(ascending=False)
# TOP 5 Services 300, 460, 510, 636, 259

REVCODE
300    53
460    47
510    18
636    12
259    10
450    10
250     7
301     6
730     5
258     4
420     4
260     4
270     4
320     4
710     3
370     3
360     3
762     3
255     2
305     2
352     2
310     2
324     2
424     2
430     2
272     1
351     1
361     1
483     1
771     1
Name: Uniq, dtype: int64